In [1]:
import argparse
import os
from epitome.models import *
from epitome.functions import *
from epitome.viz  import *

from epitome.constants import *
from epitome.motif_functions import *
import yaml
import subprocess
from timeit import default_timer as timer

/home/eecs/jahnavis/miniconda/envs/EpitomeNewEnv2/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


### Set Up

In [2]:
results_path = "results"
epitome_data_path = "data/epitome_data" 
motif_dir = "data/motif_data/"
feature_path = os.path.join(epitome_data_path, "feature_name")

# TF's being predicted
TF = "EGR1"
query_cell = 'K562' #'T47D'

train_iterations = 5000
test_iterations = 10000

In [3]:
def setup_directories(TF, results_path=results_path):
    # create user directories if they do not exist
    epitome_results_dir = os.path.join(results_path, "epitome_results")
    if not os.path.exists(epitome_results_dir):
        os.makedirs(epitome_results_dir)

    epitome_models_dir = os.path.join(results_path, "epitome_models")
    if not os.path.exists(epitome_results_dir):
        os.makedirs(epitome_models_dir)

    # Folder based on TF being predicted
    tf_results_dir = os.path.join(epitome_results_dir, TF + "_results")
    if not os.path.exists(tf_results_dir):
        os.makedirs(tf_results_dir)

    # Folder based on TF being predicted
    tf_model_dir = os.path.join(epitome_models_dir, TF + "_models")
    if not os.path.exists(tf_model_dir):
        os.makedirs(tf_model_dir)
    
    return tf_results_dir, tf_model_dir

tf_results_dir, tf_model_dir = setup_directories(TF)

### Load in Data for Epitome

In [4]:
train_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'train.npz')).toarray()
valid_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'valid.npz')).toarray()
test_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'test.npz')).toarray()
data = {Dataset.TRAIN: train_data, Dataset.VALID: valid_data, Dataset.TEST: test_data}
# all_data = np.concatenate((data[Dataset.TRAIN], data[Dataset.VALID], data[Dataset.TEST]), axis=1)

In [5]:
motifmat = np.load(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_unique_motifmat.npz"))["tf"]
motifmap = pd.read_csv(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_unique_motifmap.csv"), 
                       header=None).rename(columns={0:"Index", 1:"TF"})

In [6]:
# Determine Anchor TF's we have data for
epitome_tfs = list(motifmap["TF"].unique()) + ["DNase"]
anchor_tfs = ["CTCF", "E2F1", "EGR1", "FOXA1", "FOXA2", "GABPA", "HNF4A", "JUND", 
              "MAX", "NANOG", "REST", "TAF1"]
anchor_overlap_tfs = set(epitome_tfs).intersection(set(anchor_tfs))
len(anchor_tfs), len(anchor_overlap_tfs), anchor_overlap_tfs

(12,
 9,
 {'CTCF', 'E2F1', 'EGR1', 'FOXA1', 'GABPA', 'JUND', 'MAX', 'REST', 'TAF1'})

### Train VLP Model With Motif Data

In [7]:
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2) #10)

In [8]:
mat_ind = motifmap[motifmap["TF"].isin(anchor_overlap_tfs)]['Index'].tolist()

In [9]:
motifmat[mat_ind, :].shape

(9, 3268840)

In [11]:
model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap,
            motifmat = motifmat, 
            motifmap = motifmap,
            motif_assays = [TF])

start = timer()
model.train(train_iterations)
end = timer()
train_time = end-start
print('epitome train time %f' % train_time)

model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_motif")
model.save(model_path)

using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.TRAIN
motif_assays:  ['EGR1']
using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.VALID
motif_assays:  ['EGR1']
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(53.239796, shape=(), dtype=float32)tf.Tensor(45.314137, shape=(), dtype=float32)tf.Tensor(7.9256587, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(19.375076, shape=(), dtype=float32)tf.Tensor(13.695102, shape=(), dtype=float32)tf.Tensor(5.6799746, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(7.1557245, shape=(), dtype=float32)tf.Tensor(2.9662507, shape=(), dtype=float32)tf.Tensor(4.1894736, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(5.252386, shape=(), dtype=float32)tf.Tensor(1.8430265, shape=(), dtype=float32)tf.Tensor(3.4093595, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(13.413572, shape=(), dtype=float32)tf.Tensor(10.433568, shap

In [12]:
model_results = model.test(test_iterations, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

157it [00:20,  7.54it/s]

INFO:tensorflow:macro auROC:     0.8624020168460068
INFO:tensorflow:auPRC:     0.27509110518020485
INFO:tensorflow:GINI:     0.7248040753589121
Model auROC: 0.8624020168460068. Model auPRC: 0.27509110518020485.


In [13]:
eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'predicted_transcription_factor' : TF, #", ".join(anchor_overlap_tfs),
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'trained_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
   'motif_transcription_factors' : TF,
   'iterations_trained' : train_iterations,
   'iterations_tested' : test_iterations,
   'train_time' : train_time,
    }, 
    ignore_index=True)
eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.csv'), 
                       sep="\t")

preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.npz')
np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

In [14]:
model_results['preds_mean'].numpy()

array([[0.0089899 ],
       [0.00424215],
       [0.00695356],
       ...,
       [0.00584844],
       [0.00483397],
       [0.09490332]], dtype=float32)

In [15]:
model_results

{'preds_mean': <tf.Tensor: shape=(10000, 1), dtype=float32, numpy=
 array([[0.0089899 ],
        [0.00424215],
        [0.00695356],
        ...,
        [0.00584844],
        [0.00483397],
        [0.09490332]], dtype=float32)>,
 'preds_std': <tf.Tensor: shape=(10000, 1), dtype=float32, numpy=
 array([[0.00395101],
        [0.00108542],
        [0.00239465],
        ...,
        [0.00305609],
        [0.00160502],
        [0.04667194]], dtype=float32)>,
 'truth': <tf.Tensor: shape=(10000, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 'weights': <tf.Tensor: shape=(10000, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], dtype=float32)>,
 'assay_dict': {'EGR1': {'AUC': 0.8624020168460068,
   'auPRC': 0.27509110518020485,
   'GINI': 0.7248040753589121}},
 'auROC': 0.8624020168460068,
 'auPRC': 0.27509110518020485}

### VLP Model Without Motif Data

In [16]:
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2) #10)

In [17]:
model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap)

start = timer()
model.train(train_iterations)
end = timer()
train_time = end-start
print('epitome train time %f' % train_time)

model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_no_motif")
model.save(model_path)

using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.TRAIN
motif_assays:  None
using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.VALID
motif_assays:  None
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(51.907845, shape=(), dtype=float32)tf.Tensor(44.56864, shape=(), dtype=float32)tf.Tensor(7.339202, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(6.508755, shape=(), dtype=float32)tf.Tensor(1.2570472, shape=(), dtype=float32)tf.Tensor(5.251708, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(4.4476795, shape=(), dtype=float32)tf.Tensor(0.59245855, shape=(), dtype=float32)tf.Tensor(3.8552208, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(5.541329, shape=(), dtype=float32)tf.Tensor(2.4400764, shape=(), dtype=float32)tf.Tensor(3.1012526, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(4.443017, shape=(), dtype=float32)tf.Tensor(1.7138529, shape=(), dtype=float32)tf.Tensor(2.7291644, shape=(), dtype=float32)
epitome train time 

In [18]:
model_results = model.test(test_iterations, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

157it [00:16,  9.31it/s]

INFO:tensorflow:macro auROC:     0.867253058335861
INFO:tensorflow:auPRC:     0.2984019171488106
INFO:tensorflow:GINI:     0.7345061166717217
Model auROC: 0.867253058335861. Model auPRC: 0.2984019171488106.


In [19]:
eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'predicted_transcription_factor' : TF, #", ".join(anchor_overlap_tfs),
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'trained_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
   'iterations_trained' : train_iterations,
   'iterations_tested' : test_iterations,
   'train_time' : train_time,
    }, 
    ignore_index=True)
eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.csv'), 
                       sep="\t")

preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.npz')
np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

### Train/ Evaluate on all Anchor TF's

In [21]:
for tf in  ['E2F1', 'GABPA']:#anchor_overlap_tfs:
    # TF's being predicted
    TF = tf
    
#     if TF in ["CTCF", "JUND"]:
#         continue
    
    print("Training %s..." % TF)
    query_cell = 'K562' #'T47D'
    
    tf_results_dir, tf_model_dir = setup_directories(TF)
    
    matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
    
    # Train TF with Motif Data
    model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap,
            motifmat = motifmat, 
            motifmap = motifmap,
            motif_assays = [TF])

    start = timer()
    model.train(train_iterations)
    end = timer()
    train_time = end-start
    print('%s Motif Train Time %f' % (TF, train_time))

    model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_motif")

    model.save(model_path)
    
    # Test TF with Motif Model
    model_results = model.test(test_iterations, calculate_metrics=True)
    
    # Save Motif Model
    eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
    eval_results_df = eval_results_df.append({ 
       'predicted_transcription_factor' : TF, #", ".join(anchor_overlap_tfs),
       'query_cell' : query_cell,
       'auROC' : model_results['auROC'],
       'auPRC' : model_results['auPRC'],
       'trained_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
       'motif_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
       'iterations_trained' : train_iterations,
       'iterations_tested' : test_iterations,
       'train_time' : train_time,
        }, 
        ignore_index=True)
    eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + 
                                        '_motif.csv'), sep="\t")

    preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.npz')
    np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())
    
    # Train TF without Motif Data
    model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap)

    start = timer()
    model.train(train_iterations)
    end = timer()
    train_time = end-start
    print('%s Non-Motif Train Time %f' % (TF, train_time))

    model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_no_motif")
    model.save(model_path)
    
    # Test TF with Non-Motif Model
    model_results = model.test(test_iterations, calculate_metrics=True)
    
    eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
    eval_results_df = eval_results_df.append({ 
       'predicted_transcription_factor' : TF,
       'query_cell' : query_cell,
       'auROC' : model_results['auROC'],
       'auPRC' : model_results['auPRC'],
       'trained_transcription_factors' : TF,
       'iterations_trained' : train_iterations,
       'iterations_tested' : test_iterations,
       'train_time' : train_time,
        }, 
        ignore_index=True)
    eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.csv'), 
                           sep="\t")

    preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.npz')
    np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

Training E2F1...
using ['K562', 'HeLa-S3'] as labels for mode Dataset.TRAIN
motif_assays:  ['E2F1']
using ['K562', 'HeLa-S3'] as labels for mode Dataset.VALID
motif_assays:  ['E2F1']
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(45.529, shape=(), dtype=float32)tf.Tensor(40.207222, shape=(), dtype=float32)tf.Tensor(5.3217773, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(13.986225, shape=(), dtype=float32)tf.Tensor(10.15517, shape=(), dtype=float32)tf.Tensor(3.8310552, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(9.72357, shape=(), dtype=float32)tf.Tensor(6.84454, shape=(), dtype=float32)tf.Tensor(2.8790293, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(3.1363087, shape=(), dtype=float32)tf.Tensor(0.70637953, shape=(), dtype=float32)tf.Tensor(2.4299293, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(2.8301988, shape=(), dtype=float32)tf.Tensor(0.5962671, shape=(), dtype=float32)tf.Tensor(2.2339318, shape=(), dtype=float32)


0it [00:00, ?it/s]

E2F1 Motif Train Time 686.147521


157it [00:18,  8.65it/s]

INFO:tensorflow:macro auROC:     0.8304208416833668
INFO:tensorflow:auPRC:     0.00785415467696559
INFO:tensorflow:GINI:     0.6608416540111473


...

INFO:tensorflow:macro auROC:     0.8070505758637957
INFO:tensorflow:auPRC:     0.011481077721299419
INFO:tensorflow:GINI:     0.6141011321670005
using ['K562', 'HepG2', 'HeLa-S3', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
motif_assays:  None
using ['K562', 'HepG2', 'HeLa-S3', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.VALID
motif_assays:  None
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(62.871258, shape=(), dtype=float32)tf.Tensor(48.130913, shape=(), dtype=float32)tf.Tensor(14.740345, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(19.880205, shape=(), dtype=float32)tf.Tensor(9.781725, shape=(), dtype=float32)tf.Tensor(10.09848, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(13.626184, shape=(), dtype=float32)tf.Tensor(6.514467, shape=(), dtype=float32)tf.Tensor(7.111717, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(23.139746, shape=(), dtype=float32)tf.Tensor(17.803673, shape=(), dtype=float32)tf.Tensor(5.3360734,

0it [00:00, ?it/s]

GABPA Non-Motif Train Time 1258.165177


157it [00:30,  5.10it/s]


INFO:tensorflow:macro auROC:     0.8207110665998998
INFO:tensorflow:auPRC:     0.01918505803798979
INFO:tensorflow:GINI:     0.6414221429800951
